<div style="text-align: right;">
<a target="_blank" href="https://colab.research.google.com/github/hkaragah/hkaragah.github.io/blob/main/structure/concrete/force_moment_interaction/force_moment_interaction.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<div style="font-size: 0.8em; color: #555;">By Hossein Karagah</div>
<div style="font-size: 0.8em; color: #555;">© Copyright 2025 GNU GENERAL PUBLIC LICENSE.</div>
</div>

# Seismic Forces on Floor Diaphragms

<img src="../../../assets/img/structure/structural_dynamics/seismic_forces_on_floor_diaphragms/seismic_forces_on_floor_diaphragm_concrete_slab.jpg" width=100% alt="Frame properties">

# Seismic Forces on Floor Diaphragms



## Setup Environment

In [1]:
# !pip install comtypes

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
import comtypes.client

## Define Structural Model

### Seismic Design Parameters

* Location: San Diego, CA (Lat: 32.71576, Long: -117.163817)
* Soil Class: D-Stiff Soil ($V_{S30}=260$)
* Risk Category: II
* $PGA_{M}=0.69$
* $T_L=8$ sec
* $S_S = 1.65$
* $S_1 = 0.5$
* $S_{MS}=1.64$
* $S_{M1}=1.19$
* $S_{DS}=1.09$
* $S_{D1}=0.79$

### Loads

* Dead load: 95 psf (including 6" concrete slab)
* Live load: 50 psf (office)
* Roof live load: 20 psf
* Tributary width: 18 ft

### Model Parameters

* Lateral force resisting: Steel special moment-resisting frame
* $R=8$
* $\Omega_0=3$
* $C_d=5.5$
* $I_e=1.0$

The section used for beams and columns are as follows:

|Story|1|2|3|4|5|6|7|8|9|10|
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|Columns|W8x67<br>12 ft|W12x96<br>9 ft|W8x40<br>9 ft|W14x68<br>9 ft|W12x45<br>9 ft|W14x53<br>9 ft|W12x45<br>9 ft|W14x48<br>9 ft|W12x45<br>9 ft|W12x45<br>9 ft|
|Beam|W18x35|W18x35|W18x35|W18x35|W18x35|W18x35|W18x35|W18x35|W18x35|W18x35|

The beam's spans are 18 ft. Also the frame is assumed simply supported at the base. 

In [3]:
from dataclasses import dataclass

@dataclass
class Section:
    name: str
    self_weight: float # in lb/ft
    
    
@dataclass
class Element:
    section: Section
    length: float # in feet
    unit_dead_load: float = 0.0  # Unit dead load in lb/ft, optional
    
    @property
    def weight(self) -> float:
        """Calculate the weight of the element."""
        return (self.section.self_weight + self.unit_dead_load) * self.length
  
    def __repr__(self):
        load_str = f", DL={self.unit_dead_load:.1f} lb/ft" if self.unit_dead_load != 0 else ""
        return f"{self.section.name}, L={self.length} ft{load_str}"
    
@dataclass
class Story:
    name: str
    beam: Element
    columns_below: Element
    n_col_below: int
    columns_above: Element = None  # Optional, for the top story
    n_col_above: int = 0  # Optional, for the top story
    
    @property
    def seismic_weight(self) -> float:
        """Calculate the effective seismic weight of the story.
        This includes the weight of the beam, columns below, and optionally columns above."""
        floor_weight = self.beam.weight
        columns_below_weight = self.columns_below.weight * self.n_col_below / 2
        columns_above_weight = self.columns_above.weight * self.n_col_above / 2 if self.columns_above else 0.0
        return floor_weight + columns_below_weight + columns_above_weight
    
    @property
    def height(self) -> float:
        """Calculate the height of the story based on the columns below."""
        return self.columns_below.length
    
    def __repr__(self):
        return (
            f"{self.name}:"
            f"beam ({self.beam}), "
            f"column below ({self.n_col_below}-{self.columns_below}), "
            f"columns above ({self.n_col_above}-{self.columns_above if self.columns_above else None})"
        )
        
class Building:
    def __init__(self, name: str, R: float, Cd: float, Omega_0: float, Ie: float, period: float = None):
        self.name = name
        self.R = R
        self.Cd = Cd
        self.Omega_0 = Omega_0
        self.Ie = Ie
        self.period = period  # Optional, can be set later
        self.stories: list[Story] = []
        
    def add_story(self, story: Story):
        """Add a story to the building."""
        self.stories.append(story)
        
    def seismic_weight(self) -> float:
        """Calculate the total effective seismic weight of the building
        by adding the effective weights of all stories.
        """
        return sum(story.seismic_weight for story in self.stories)
    
    @property
    def story_heights(self) -> list:
        """Get the elevations of all stories in the building."""
        return [story.height for story in self.stories]
    
    @property
    def story_elevations(self) -> list:
        """Get the elevations of all stories in the building."""
        elevations = []
        current_elevation = 0.0
        for story in self.stories:
            current_elevation += story.height
            elevations.append(current_elevation)
        return elevations
    
    @property
    def story_weights(self) -> list:
        """Get the effective seismic weights of all stories in the building."""
        return [story.seismic_weight for story in self.stories]
    
    def _k(self) -> float:
        if self.period is None:
            raise ValueError("Period must be set to calculate k.")
        return 1.0 + (self.period - 0.5) / (2.5 - 0.5)
    
    @property
    def C_vx(self) -> list[float]:
        """Calculate the vertical seismic force distribution factor for each story
        per ASCE 7-22 §12.8.3.
        """
        if not self.stories:
            raise ValueError("No stories in the building to calculate C_vx.")
        if self.period is None:
            raise ValueError("Period must be set to calculate C_vx.")
        
        k = self._k()
        wx = np.array(self.story_weights)
        hx = np.array(self.story_elevations)

        return wx * hx**k / np.sum(wx * hx**k)
    
    
    def __repr__(self):
        # Create header
        header = f"\nBuilding: {self.name}\n"
        header += f"R: {self.R}, Cd: {self.Cd}, Omega_0: {self.Omega_0}, Ie: {self.Ie}\n"
        header += "-" * 45 + "\n"
        header += "Story  Weight(lb)  Height(ft)  Elevation(ft)\n"
        header += "-" * 45 + "\n"
        
        # Build table rows in reverse order
        rows = []
        for i, (story, height, elev) in enumerate(zip(
            reversed(self.stories),
            reversed(self.story_heights), 
            reversed(self.story_elevations)
        )):
            rows.append(
                f"{len(self.stories)-i:^5d}  {story.seismic_weight:^10.1f}  "
                f"{height:^10.1f}  {elev:^12.1f}"
            )
            
        # Add dashed line and total weight
        rows.append("-" * 45)
        rows.append(f"{'Total':^5s}  {self.seismic_weight():^10.1f}")
            
        return header + "\n".join(rows)    
    
@dataclass
class SeismicParams:
    PGA_M: float
    T_L: float
    S_S: float
    S_1: float
    S_MS: float
    S_M1: float
    
    @property
    def S_DS(self) -> float:
        return (2/3) * self.S_MS
    
    @property
    def S_D1(self) -> float:
        return (2/3) * self.S_M1
    
    def __str__(self):
        return (
            f"SeismicParams:\n"
            f"PGAₘ = {self.PGA_M}\n"
            f"T = {self.T_L}\n"
            f"Sₛ = {self.S_S}\n"
            f"S₁ = {self.S_1}\n"
            f"Sₘₛ = {self.S_MS}\n"
            f"Sₘ₁ = {self.S_M1}\n"
            f"Sdₛ = {self.S_DS:.2f}\n"
            f"Sd₁ = {self.S_D1:.2f}"
        )

    def __repr__(self):
        return self.__str__()

In [41]:

unit_dead_load = 95 # psf
tirbutary_width = 18 # ft
beam_dead_load = unit_dead_load * tirbutary_width # lb/ft

# Height of each story in ft
story_heights = [12, 9, 9, 9, 9, 9, 9, 9, 9, 9]

# Total height of the frame
total_height = sum(story_heights)

# Define beam properties
W18x35 = Section(name="W18x35", self_weight=35)
B1 = Element(section=W18x35, length=18, unit_dead_load=beam_dead_load)
beams = [B1] * 10

# Define column properties
W8x67 = Section(name="W8x67", self_weight=67)
W12x96 = Section(name="W12x96", self_weight=96)
W8x40 = Section(name="W8x40", self_weight=40)
W14x68 = Section(name="W14x68", self_weight=68)
W12x45 = Section(name="W12x45", self_weight=45)
W14x53 = Section(name="W14x53", self_weight=53)
W14x48 = Section(name="W14x48", self_weight=48)
column_sections = [W8x67, W12x96, W8x40, W14x68, W12x45, W14x53, W12x45, W14x48, W12x45, W12x45]
columns = []
for section, height in zip(column_sections, story_heights):
    columns.append(Element(section=section, length=height))

# Define stories with beams and columns
stories: list[Story] = []
story_count = 0
for i, (beam, column_below) in enumerate(zip(beams, columns)):
    story_count += 1
    stories.append(Story(
        name=f"Story {story_count}",
        beam=beam,
        columns_below=column_below,
        n_col_below=2,
        columns_above=columns[i+1] if i < len(columns)-1 else None,
        n_col_above=2 if i < len(columns)-1 else 0
    ))
    print(stories[-1])


Story 1:beam (W18x35, L=18 ft, DL=1710.0 lb/ft), column below (2-W8x67, L=12 ft), columns above (2-W12x96, L=9 ft)
Story 2:beam (W18x35, L=18 ft, DL=1710.0 lb/ft), column below (2-W12x96, L=9 ft), columns above (2-W8x40, L=9 ft)
Story 3:beam (W18x35, L=18 ft, DL=1710.0 lb/ft), column below (2-W8x40, L=9 ft), columns above (2-W14x68, L=9 ft)
Story 4:beam (W18x35, L=18 ft, DL=1710.0 lb/ft), column below (2-W14x68, L=9 ft), columns above (2-W12x45, L=9 ft)
Story 5:beam (W18x35, L=18 ft, DL=1710.0 lb/ft), column below (2-W12x45, L=9 ft), columns above (2-W14x53, L=9 ft)
Story 6:beam (W18x35, L=18 ft, DL=1710.0 lb/ft), column below (2-W14x53, L=9 ft), columns above (2-W12x45, L=9 ft)
Story 7:beam (W18x35, L=18 ft, DL=1710.0 lb/ft), column below (2-W12x45, L=9 ft), columns above (2-W14x48, L=9 ft)
Story 8:beam (W18x35, L=18 ft, DL=1710.0 lb/ft), column below (2-W14x48, L=9 ft), columns above (2-W12x45, L=9 ft)
Story 9:beam (W18x35, L=18 ft, DL=1710.0 lb/ft), column below (2-W12x45, L=9 ft), 

In [42]:
# Create the building and add stories
building = Building(name="modal_analysis_10_story", R=8.0, Cd=5.5, Omega_0=3.0, Ie=1.0)
for story in stories:
    building.add_story(story)
    
print(building)


Building: modal_analysis_10_story
R: 8.0, Cd: 5.5, Omega_0: 3.0, Ie: 1.0
---------------------------------------------
Story  Weight(lb)  Height(ft)  Elevation(ft)
---------------------------------------------
 10     31815.0       9.0          93.0    
  9     32220.0       9.0          84.0    
  8     32247.0       9.0          75.0    
  7     32247.0       9.0          66.0    
  6     32292.0       9.0          57.0    
  5     32292.0       9.0          48.0    
  4     32427.0       9.0          39.0    
  3     32382.0       9.0          30.0    
  2     32634.0       9.0          21.0    
  1     33078.0       12.0         12.0    
---------------------------------------------
Total   323634.0 


In [43]:
seismic = SeismicParams(
    PGA_M=0.69,  # Peak Ground Acceleration in g
    T_L=8.0,    # Long period transition in seconds
    S_S=1.65,    # Short period spectral acceleration
    S_1=0.5,    # Long period spectral acceleration
    S_MS=1.64,   # Site coefficient for short period
    S_M1=1.19    # Site coefficient for long period
)

seismic

SeismicParams:
PGAₘ = 0.69
T = 8.0
Sₛ = 1.65
S₁ = 0.5
Sₘₛ = 1.64
Sₘ₁ = 1.19
Sdₛ = 1.09
Sd₁ = 0.79

In [44]:
def get_approx_period(building: Building) -> float:
    """Calculate the approximate fundamental period of the building."""
    total_height = building.story_elevations[-1]
    C_t = 0.028  # ASCE 7-22 Table 12.8-2 coefficient
    x = 0.8      # ASCE 7-22 Table 12.8-2 exponent
    T_a = C_t * (total_height ** x)
    return T_a

def get_upper_bound_period(building: Building) -> float:
    """Calculate the upper bound of the fundamental period."""
    T_a = get_approx_period(building)
    C_u = 1.4  # ASCE 7-22 §12.8.2 coefficient
    T_max = C_u * T_a
    return T_max

def get_period(building: Building) -> float:
    if building.period is None:
        building.period = get_approx_period(building)
    else:
        T_max = get_upper_bound_period(building)
        if building.period > T_max:
            building.period = T_max
    return building.period

T_a = get_approx_period(building)
print(f"Approximate fundamental period:\nT_a = {T_a:.3f} sec")

T_max = get_upper_bound_period(building)
print(f"\nUpper bound of the fundamental period:\nT_max = {T_max:.3f} sec")

Approximate fundamental period:
T_a = 1.052 sec

Upper bound of the fundamental period:
T_max = 1.473 sec


In [45]:
def get_Cs(building: Building, seismic: SeismicParams) -> float:
    """Calculate seismic response coefficient Cs per ASCE 7-22"""
    T = _get_design_period(building)
    C_s_max = _get_Cs_max(building, seismic, T)
    C_s_min = _get_Cs_min(building, seismic)
    C_s = _get_base_Cs(building, seismic)
    
    return max(min(C_s, C_s_max), C_s_min)

def _get_design_period(building: Building) -> float:
    """Get design period T"""
    T_a = get_approx_period(building)
    T_max = get_upper_bound_period(building) 
    return min(building.period, T_max) if building.period else T_a

def _get_Cs_max(building: Building, seismic: SeismicParams, T: float) -> float:
    """Calculate maximum seismic coefficient"""
    R_Ie = building.R / building.Ie
    if T <= seismic.T_L:
        # ASCE 7-22 Equation 12.8-4
        return seismic.S_D1 / (R_Ie * T)
    return seismic.S_D1 * seismic.T_L / (R_Ie * T**2)

def _get_Cs_min(building: Building, seismic: SeismicParams) -> float:
    """Calculate minimum seismic coefficient"""
    R_Ie = building.R / building.Ie
    C_s_min = max(0.044 * seismic.S_DS * building.Ie, 0.01)
    if seismic.S_1 >= 0.6:
        C_s_min = max(C_s_min, 0.5 * seismic.S_1 / R_Ie)
    return C_s_min

def _get_base_Cs(building: Building, seismic: SeismicParams) -> float:
    """Calculate base seismic coefficient"""
    return seismic.S_DS / (building.R / building.Ie)


In [46]:
# Set the building period to the priod of the ETABS model
building.period = 2.462 # seconds
T = get_period(building)  # Use the calculated period
print(f"\nBuilding period T = {T:.3f} sec")

# Calculate seismic response coefficient per ASCE 7-22 §12.8.1.1
C_s= get_Cs(building, seismic)
print(f"Seismic response coefficient Cs = {C_s:.3f}")
print(f"Total seismic weight W = {building.seismic_weight()/1e3:.1f} kips")
# Calculate seismc base shear per ASCE 7-22 §12.8.1
V = C_s * building.seismic_weight()
print(f"Seismic base shear V = Cs * W = {V/1e3:.1f} kips")

C_vx = building.C_vx
F_x = C_vx * V
# compute accumulative sum
V_x = np.cumsum(F_x[::-1])[::-1]  # Reverse the order for cumulative sum

# Print header
print("\nStory\tC_vx\tF_x (kips)\tV_x (kips)")
print("-" * 45)
# Print values in table format
for i, (cv, f, v) in enumerate(zip(C_vx[::-1], F_x[::-1], V_x[::-1])):
    print(f"{len(building.stories)-i:4d}\t{cv:.3f}\t{f/1e3:8.1f}\t{v/1e3:8.1f}")
print("-" * 45)
print(f"{'Total':^5s}\t\t{F_x.sum()/1e3:^11.1f}")


Building period T = 1.473 sec
Seismic response coefficient Cs = 0.067
Total seismic weight W = 323.6 kips
Seismic base shear V = Cs * W = 21.8 kips

Story	C_vx	F_x (kips)	V_x (kips)
---------------------------------------------
  10	0.212	     4.6	     4.6
   9	0.185	     4.0	     8.6
   8	0.156	     3.4	    12.0
   7	0.129	     2.8	    14.9
   6	0.104	     2.3	    17.1
   5	0.081	     1.8	    18.9
   4	0.059	     1.3	    20.2
   3	0.040	     0.9	    21.0
   2	0.024	     0.5	    21.6
   1	0.011	     0.2	    21.8
---------------------------------------------
Total		   21.8    


## ETABS Model

### Setup

In [38]:
# Create API helper object
helper = comtypes.client.CreateObject("ETABSv1.Helper")
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
ETABSObject = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
ETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject")

# Start ETABS application
ETABSObject.ApplicationStart()

# Create and initialize SapModel object
SapModel = ETABSObject.SapModel
SapModel.InitializeNewModel()

0

### Run ETABS Model Analysis

In [39]:
# Open the model
etabs_model = r"C:\Users\hkara\OneDrive\Desktop\sap model\modal_analysis_10_story.edb"
SapModel.File.OpenFile(etabs_model)

# Save the model under new name
etabs_model = r"C:\Users\hkara\OneDrive\Desktop\sap model\modal_analysis_10_story.edb"
SapModel.File.Save(etabs_model)

# Run the analysis
SapModel.Analyze.RunAnalysis()

0

### Modal Analysis Results

In [47]:
# Make sure to deselect all cases and combos for output
SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()

# Set the load case for output to "MODAL"
SapModel.Results.Setup.SetCaseSelectedForOutput("MODAL")

# Set units to US Customary (kip, ft, sec)
SapModel.SetPresentUnits(4)  

# Retrieve modal periods
out = SapModel.Results.ModalPeriod()

# Extract modal results and reshaope into a DataFrame
modal_df = pd.DataFrame({
    "period_sec": np.round(out[4], 3),
    "frequency_Hz": np.round(out[5], 3),
    "circular_frequency_rad_sec": np.round(out[6], 3),
    "eigenvalue": np.round(out[7], 3)
})

modal_df

,period_sec,frequency_Hz,circular_frequency_rad_sec,eigenvalue
0,2.462,0.406,2.552,6.513
1,0.762,1.312,8.244,67.970
2,0.393,2.545,15.990,255.665
3,0.254,3.943,24.772,613.639
4,0.179,5.591,35.129,1234.066
5,0.134,7.468,46.922,2201.673
6,0.107,9.309,58.490,3421.064
7,0.088,11.381,71.509,5113.603
8,0.077,12.980,81.553,6650.963
9,0.068,14.710,92.425,8542.443


In [14]:
# Name = "ALL"
# GroupElm = 1
# ItemTypeElm = 2
# NumberResults = 1
# Obj = []
# Elm = []
# LoadCase = ["MODAL"]
# StepType = []
# StepNum = []
# U1, U2, U3 = [0], [1], [0]
# R1, R2, R3 = [0], [0], [0]
# out = SapModel.Results.ModeShape(
#     Name, 
#     GroupElm, 
#     NumberResults, 
#     Obj, 
#     Elm, 
#     LoadCase, 
#     StepType, 
#     StepNum, 
#     U1, 
#     U2, 
#     U3, 
#     R1, 
#     R2, 
#     R3
# )
# out = SapModel.Results.ModeShape()
# out

In [15]:
# # Deselect previous load combinations
# SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()

# # Get load combination names
# out = SapModel.RespCombo.GetNameList()
# combinations = list(out[1])

# # 3) Select each combo for output
# for combo in combinations:
#     SapModel.Results.Setup.SetComboSelectedForOutput(combo, True)
    
# # Get base reactions for all load combinations
# out = SapModel.Results.BaseReact()

# # Extract the relevant data from the output
# reaction_df = pd.DataFrame({
#     'Load Case': out[1],
#     'FX': np.round(out[4], 3),
#     'FY': np.round(out[5], 3), 
#     'FZ': np.round(out[6], 3),
#     'MX': np.round(out[7], 3),
#     'MY': np.round(out[8], 3),
#     'MZ': np.round(out[9], 3)
# })

# reaction_df


### Get Story and Diaphragm Forces

In [48]:
out = SapModel.Story.GetStories()

# out[1] = Story name
# out[2] = Story elevation
# out[3] = Story height

story_names = out[1]
story_elevations = out[2]
story_heights = out[3]

In [49]:
# List of available tables

NumberTables = 1
TableKey = []
TableName = []
ImportType = []

out = SapModel.DatabaseTables.GetAvailableTables(NumberTables, TableKey, TableName, ImportType)

number_of_tables = out[0]
table_names = out[1]

# Filter table names to include only those related to "Story"
table_names = [name for name in table_names if "Story" in name]
table_names

['Mass Summary by Story',
 'Material List by Story',
 'Story Definitions',
 'Story Drifts',
 'Story Forces',
 'Story Max Over Avg Displacements',
 'Story Max Over Avg Drifts',
 'Story Stiffness',
 'Tower and Base Story Definitions']

In [51]:
# Make sure no cases *or* combos are selected
SapModel.DatabaseTables.SetLoadCasesSelectedForDisplay([])
SapModel.DatabaseTables.SetLoadCombinationsSelectedForDisplay([])

# Select only the "Exh" case
SapModel.DatabaseTables.SetLoadCasesSelectedForDisplay(["Eh"])

# Pull the Story Forces table
TableKey = 'Story Forces'
FieldKeyList = []    # empty = all fields
GroupName = ""    # empty = all objects
TableVersion = 1
FieldsKeysIncluded = []
NumberRecords = 1
TableData = []

out = SapModel.DatabaseTables.GetTableForDisplayArray(
    TableKey,
    FieldKeyList,
    GroupName,
    TableVersion,
    FieldsKeysIncluded,
    NumberRecords,
    TableData
)

column_names = out[2]
n_rows       = out[3]
flat_data    = out[4]

# Reshape into a DataFrame
arr = np.array(flat_data).reshape(n_rows, len(column_names))
story_force_df = pd.DataFrame(arr, columns=column_names)

# Convert any numeric columns
for col in story_force_df.columns:
    try:
        story_force_df[col] = pd.to_numeric(story_force_df[col])
    except ValueError:
        continue

# Filter to only those rows where Location == "Top"
story_force_top = story_force_df[story_force_df["Location"] == "Bottom"].reset_index(drop=True)
story_force_top


,Story,OutputCase,CaseType,Location,P,VX,VY,T,MX,MY
0,Story10,Eh,LinStatic,Bottom,0,-4.428,0,0,0,-40.5479
1,Story9,Eh,LinStatic,Bottom,0,-8.234,0,0,0,-117.0833
2,Story8,Eh,LinStatic,Bottom,0,-11.450,0,0,0,-224.7426
3,Story7,Eh,LinStatic,Bottom,0,-14.109,0,0,0,-359.6490
4,Story6,Eh,LinStatic,Bottom,0,-16.248,0,0,0,-516.2467
5,Story5,Eh,LinStatic,Bottom,0,-17.905,0,0,0,-691.2394
6,Story4,Eh,LinStatic,Bottom,0,-19.121,0,0,0,-880.2711
7,Story3,Eh,LinStatic,Bottom,0,-19.945,0,0,0,-1080.8802
8,Story2,Eh,LinStatic,Bottom,0,-20.430,0,0,0,-1292.1972
9,Story1,Eh,LinStatic,Bottom,0,-20.641,0,0,0,-1616.8206


The calculated fundamental period is less than the ASCE 7 upper bound and I can use it in the ETABS model, but for simlicity I am going to keep using the ASCE 7 approximate period in the model.

## Modal Analysis

## Seismic Forces



---

### Story Forces



---

### Diaphragm Forces




## Compare Seismic Forces with Modal Shapes